-----------------------------------------**FIRST PYTHON MOVIE PROJECT (WEB SCRAPPING)**-------------------------------------------

**INSTALLING ALL LIBRARY**

In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


**lxml : TO RUN THE CODE IN HTML LANGUAGES**

In [7]:
pip install lxml

In [1]:
import pandas as pd
import lxml
import requests
from bs4 import BeautifulSoup

In [3]:
# Define headers to simulate a browser visit and set the default language to English
headers = {
    'Accept-Language': 'en-US,en;q=0.5',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}


# Define base URLs
base_url = "https://www.themoviedb.org"
Page_url = "https://www.themoviedb.org/movie?page="

#check website status code 
requests.get("https://www.themoviedb.org",headers=headers)
requests.get("https://www.themoviedb.org/movie?page=",headers=headers)

# Initialize an empty list to store URLs for pagination
lst=[]

# Generate URLs for the first 100 pages of movie listings
for i in range(1,101):
    lst.append(Page_url+str(i))
    
# Initialize a list to store all movie details
all_movies_data=[]

# Loop through each URL in the URL list   
for url in lst:
        # Get the source code of the current page
        source_code = requests.get(url,headers=headers).text
        # Parse the source code using BeautifulSoup
        converted_data = BeautifulSoup(source_code, 'lxml')
        
        # Find all movie detail containers on the page
        movie_details = converted_data.find_all('div', class_='card style_1')
        
        # Loop through each movie detail container
        for pic in movie_details:
            img = pic.find('img')
            Movie_Name = img.attrs['alt']
            Ratings = pic.find('div',class_='user_score_chart').get('data-percent')
            Release_Date = pic.find('p').text
            
            # Extract the link for each movie
            movie_link = base_url + pic.find('a')['href']
        
            # Navigate to the individual movie page
            response = requests.get(movie_link,headers=headers).text
            soup = BeautifulSoup(response,'lxml')
            
            # Extract duration
            Durations_M = soup.find('span', class_="runtime").text.strip() if soup.find('span', class_="runtime") else "N/A"
             
            # Extract genres
            Movie_Genre = soup.find("span", class_="genres").text.replace("\n", "").replace("\xa0", " ") if soup.find("span", class_="genres") else "N/A"
            
            # Extract directors names
            Directors_Name = []
            directors = soup.find_all('li', class_='profile')
            for director in directors:
                if director.find('p', class_='character').text == "Director":
                    Directors_Name.append(director.find('p').text)
                    Directors_Name = Directors_Name if Directors_Name else "N/A"

            # Store the details in a dictionary
            movie_data = {
                "Movie_Name": Movie_Name,
                "Release_Date": Release_Date,
                "Ratings": Ratings,
                "Durations_M": Durations_M,
                "Movie_Genre": Movie_Genre,
                "Directors_Name": Directors_Name
            }
            
            # Append the dictionary to the list of all movies
            all_movies_data.append(movie_data)

In [4]:
# Convert the list of movie details to a DataFrame
df = pd.DataFrame(all_movies_data)

In [5]:
df

,Movie_Name,Release_Date,Ratings,Durations_M,Movie_Genre,Directors_Name
0,Inside Out 2,"Jun 14, 2024",77,1h 37m,"Animation, Family, Adventure, Comedy, Drama",[]
1,Despicable Me 4,"Jul 03, 2024",75,1h 35m,"Animation, Family, Comedy, Action",[Chris Renaud]
2,The Garfield Movie,"Apr 30, 2024",73,1h 41m,"Animation, Comedy, Family, Adventure",[Mark Dindal]
3,Furiosa: A Mad Max Saga,"May 22, 2024",77,2h 29m,"Action, Adventure, Science Fiction",[]
4,Kingdom of the Planet of the Apes,"May 08, 2024",71,2h 25m,"Science Fiction, Adventure, Action",[Wes Ball]
...,...,...,...,...,...,...
1995,Harry Potter 20th Anniversary: Return to Hogwarts,"Jan 01, 2022",73,1h 43m,Documentary,"[Giorgio Testi, Eran Creevy, Casey Patterson, ..."
1996,Young Mother,"Jul 31, 2013",50,1h 23m,"Comedy, Romance",[Kong Ja-kwan]
1997,Step Up All In,"Jul 16, 2014",68,1h 52m,"Romance, Drama, Music",[Trish Sie]
1998,The Boys in the Boat,"Dec 25, 2023",74,2h 4m,"Drama, History",[George Clooney]


In [6]:
#Converting the data into excel sheet
df.to_excel('Movies_Data.xlsx')